<a href="https://colab.research.google.com/github/MUHAMMADFAZEEL01/Pinecone-Gemini-AgenticAI/blob/main/PROJECT_02_(RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain langchain-community langchain_google_genai langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.0 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
import langchain
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/ragproject-446611-8c78fb8b1c9a.json"


In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")



In [5]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key = os.environ.get('PINECONE_API_KEY'))
spec=ServerlessSpec(cloud="aws", region="us-east-1")

In [ ]:

if 'rag' not in pc.list_indexes():
  pc.create_index(
      name =  'rag' ,
      dimension =768,
      metric ='cosine',
      spec = spec
  )


In [6]:
index = pc.Index('rag')

In [7]:
pc.describe_index('rag')

{'deletion_protection': 'disabled',
 'dimension': 768,
 'host': 'rag-asi64lk.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'rag',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [8]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [9]:
!pip install --upgrade --quiet pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.1 MB/s eta 0:00:00


In [10]:
from langchain_community.document_loaders import PyPDFLoader

filepdf= (
    '/content/Strategic Mindset  A 7-Day Plan to Identify What Matters and Create a Strategy that Works (Productivity Series Book 4) (Thibaut  Meurisse) (Z-Library).pdf'
)
loader = PyPDFLoader(filepdf)


In [11]:
documents =loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [40]:
from uuid import uuid4
ids = []
for doc in docs:
    # Generate vector for the document
    vector = embeddings.embed_query(doc.page_content)
    metadata = {"source": doc.metadata.get("source", "unknown")}
    unique_id = str(uuid4())
    ids.append(unique_id)
    index.upsert([(unique_id, vector, metadata)])  # (id, vector, metadata)



a82433d3-f8eb-4782-bbb2-35e3fa14b1fd
382dc0b2-39bd-44cd-a3b9-85413eb8f1ee
43022475-dd7e-4dd4-b3b0-250cf39e0970
0e1543c2-c1f4-42fe-a4bf-cc71791c4512
dd95adf8-721c-47bf-8002-864496326f32
857ff471-683c-4d12-b385-f2c3e56aa714
dee86da7-68bc-4dea-850e-5a172a1433d9
4b70e284-5514-484f-823f-637be9b83877
742287cf-d145-41a6-a06f-4aad2f1c3df5
83a148eb-51ab-474b-bdc1-cb2091987f93
7c3c1801-482c-4513-867c-92140f975485
bd3086b6-40ad-4e19-b42f-bfbc5b224bb4
d8edce64-32d6-4349-a2bf-fbe812b2d829
32c43ec1-e4c3-40a1-8075-82352e74c824
12ded896-cf3e-4f68-9335-a5a2c6686528
0382e28c-0b0d-4cf8-a5a6-abe99d8f1c40
aa683a5f-81a7-4039-a237-ba34472fae6a
55e9d0b1-8e52-4ae4-a1c8-ccdc183357da
c9aefb29-3935-4197-8487-cfbc3f57a1bc
86dd062e-7975-42e0-ba90-19a649795b66
66107fe8-44a1-4632-88cb-9c28efb615cb
72b9362e-b6f4-45f2-a095-db0f69a658c3
fdf24505-5197-4d9b-a246-54828f7fb4f8
2d4aaec3-bebf-4392-a665-48b8561aea8d
7a59b2e4-6e88-4c00-8df3-39ff2e8bdea9
76daf449-9a26-46ea-8ffa-2c4d41758eb4
86428503-6e18-45c8-9fa0-d82276c60fee
9

In [36]:
for id in ids:
  print(id)

13671aa2-3667-4d1e-b735-1f4f72a42fab
323eef14-879d-45e7-ad87-9b103e3f8c75
6dd8253f-e766-490f-ba47-7eba9b936e5a
6c4fc3b6-519c-41cd-8cda-49bd727691d9
004022ba-57ca-4e91-b6c0-6206589ae042
7b777c93-1642-4706-b21a-28693f25a41c
c28ebcc3-2a49-498f-9747-909a7e048f57
67bf4c18-6025-44f4-b622-3897cc24e06f
f6d17304-5733-431d-b86f-86f5338783f2
a8920804-c5c8-4592-ab41-2b3ce8be839c
55ed6c9a-71fd-48b5-9843-5c3be0a10bdb
ea68ec1c-e9e7-4992-97d4-04617457c2a8
3ae2aeab-dc5a-4448-b220-d1da70a59b3c
b2622124-1667-4475-b233-14c0b48a0738
f69d5581-21f0-41fe-b31f-a77234222fd7
036c6dd5-a8d3-4765-831c-120633a4b2d2
5f5c117e-bf64-4b6e-b331-91ab193abba9
505b9b30-cea0-41d7-a435-de724a097636
c1ee3c2a-7343-4170-8e24-733bf8230737
f0fe5952-c654-44bd-b1fa-d6d6d3fdff59
f1007592-4409-4cb8-a629-72fdd8be27d7
59d08a03-f3d2-4e60-9f5d-72ec1badad8d
2996e55e-4e68-4780-a9ae-7e2f2ef21cd8
03ebd0e2-0d73-4176-818f-d54dd3fbf0a6
75b59a3e-3341-4e27-bfa5-6a2e300cb699
3903bd32-fe66-40bc-9e63-3c94c3e117e3
5efae3df-159d-4865-976e-07c1414aab22
c

In [37]:
docs[0]

Document(metadata={'source': '/content/Strategic Mindset  A 7-Day Plan to Identify What Matters and Create a Strategy that Works (Productivity Series Book 4) (Thibaut  Meurisse) (Z-Library).pdf', 'page': 1}, page_content='STRATEGIC MINDSET\nA 7-DAY PLAN TO IDENTIFY WHAT MATTERS AND\nCREATE A STRATEGY THAT WORKS\nTHIBAUT MEURISSE')

In [39]:
# Pinecone: Fetch by ID
result = index.fetch(ids=["13671aa2-3667-4d1e-b735-1f4f72a42fab"])
print(result)


{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'13671aa2-3667-4d1e-b735-1f4f72a42fab': {'id': '13671aa2-3667-4d1e-b735-1f4f72a42fab',
                                                      'metadata': {'source': '/content/Strategic '
                                                                             'Mindset  '
                                                                             'A '
                                                                             '7-Day '
                                                                             'Plan '
                                                                             'to '
                                                                             'Identify '
                                                                             'What '
                                                                             'Matters '
                                                                   

In [14]:
from langchain_pinecone import PineconeVectorStore

In [15]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [17]:
vector_store = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    text_key="text",
)

In [18]:
results = vector_store.similarity_search(
    "purpose of this book?",
    k=2,
)



In [19]:
from IPython.display import Markdown, display
for res in results:
     p = res.page_content
     display(Markdown(p))




Then, using the content from this book, develop the mindset of the strategic
producer. Whenever you face a new task, ask yourself whether it’s really
the best use of your time. If you do this consistently, over time, you’ll
become one of the best strategists you know. More importantly, you’ll reach
most of your goals, even the ones that seem difficult or perhaps impossible
right now.
It’s now up to you to perform the work and achieve the results you aspire
to. I can’t do the work for you, but I can wish you all the best in your future
endeavors.
Thibaut Meurisse.

Then, using the content from this book, develop the mindset of the strategic
producer. Whenever you face a new task, ask yourself whether it’s really
the best use of your time. If you do this consistently, over time, you’ll
become one of the best strategists you know. More importantly, you’ll reach
most of your goals, even the ones that seem difficult or perhaps impossible
right now.
It’s now up to you to perform the work and achieve the results you aspire
to. I can’t do the work for you, but I can wish you all the best in your future
endeavors.
Thibaut Meurisse.

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(api_key=os.environ.get('GEMINI_API_KEY'),temperature =0, model="gemini-1.5-flash", max_retries=2)

In [21]:
def main(query,vector_store):

  results = vector_store.similarity_search(query,k=3)
  context = '\n'.join([res.page_content for res in results])

  response = model.invoke( f" You're AI asistence in the below context is given please give me answer of the query in which is context: {context} query: {query} ")
  return response

question =str( input(" Write a question which is realted to the Strategic Mindset  A 7-Day Plan"))
result = main(question,vector_store)
display(Markdown(result.content))

 Write a question which is realted to the Strategic Mindset  A 7-Day Plandefine this book in ten points


Based on the provided text, here's a ten-point definition of the book, focusing on its core message:

1. **Focuses on strategic production:** The book's central theme is developing the mindset of a strategic producer.

2. **Prioritizes time management:**  A key element is evaluating tasks based on their value and time efficiency.

3. **Advocates for strategic thinking:** It encourages readers to become better strategists by consistently analyzing task value.

4. **Emphasizes goal achievement:** The book aims to help readers achieve their goals, even challenging ones.

5. **Promotes self-reliance:** It stresses that the reader must perform the work themselves; the book provides the framework, not the execution.

6. **Offers a practical approach:** The core concept—evaluating task value—is a readily applicable strategy.

7. **Inspires confidence:** It suggests that consistent application of the method will lead to significant success.

8. **Is action-oriented:** The book is less about theory and more about practical application and achieving results.

9. **Has a motivational tone:** The concluding remarks express support and encouragement for the reader's endeavors.

10. **Authored by Thibaut Meurisse:** The author's name is clearly stated.


Note:  This definition is inferred solely from the repeated excerpt.  A more complete definition would require access to the full book's content.
